In [186]:
# !pip install selenium 
# !pip install beautifulsoup4
#
#!pip install webdriver-manager
#!pip install lxml
#!pip install spacy
#!python3 -m spacy download en_core_web_lg
#!python3 -m pip install "pymongo[srv]"

Defaulting to user installation because normal site-packages is not writeable


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import time
import spacy
nlp = spacy.load('en_core_web_lg')
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the MongoDB URI
passwordL = os.getenv('LINKEDIN_PASSWORD')


# Creating a webdriver instance
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# This instance will be used to log into LinkedIn

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)

# entering username
username = driver.find_element(By.ID, "username")

# In case of an error, try changing the element
# tag used here.

# Enter Your Email Address
username.send_keys("rohit108180@gmail.com") 

# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element 
# tag used here.

# Enter Your Password
pword.send_keys(passwordL)	 

# Clicking on the log in button
# Format (syntax) of writing XPath --> 
# //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()
# In case of an error, try changing the
# XPath used here.

# profile_url = "https://www.linkedin.com/feed"
 
# driver.get(profile_url)  




# SETUP THE MONGO_CONNECTION
uri = mongo_uri if mongo_uri else "mongodb://localhost:27017"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [6]:
start = time.time()

# will be used in the while loop
initialScroll = 0
finalScroll = 1000

while True:
	driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll})")
	# this command scrolls the window starting from
	# the pixel value stored in the initialScroll 
	# variable to the pixel value stored at the
	# finalScroll variable
	initialScroll = finalScroll
	finalScroll += 1000

	# we will stop the script for 3 seconds so that 
	# the data can load
	time.sleep(3)
	# You can change it as per your needs and internet speed

	end = time.time()

	# We will scroll for 20 seconds.
	# You can change it as per your needs and internet speed
	if round(end - start) > 400:
		break


In [7]:

src = driver.page_source
 
# Now using beautiful soup
soup = BeautifulSoup(src, 'lxml')
# f = open("feed.txt", "w")
# f.write(src)
# f.close()

In [8]:
# fie-impression-container
feeds = soup.find_all('div', {'id': 'fie-impression-container'})

In [9]:
posts = [];
# getting the names and info about of POSTER
for feed in feeds:
    poster = feed.find('span', {'class': 'update-components-actor__name'})
    if poster :
        poster = poster.find('span', {'class': 'visually-hidden'}).get_text()
        
    
    poster_bio = feed.find('span', {'class': 'update-components-actor__description'})
    if poster_bio :
        poster_bio = poster_bio.find('span', {'class': 'visually-hidden'}).get_text()
    if poster == "LinkedIn for Marketing" or poster == "poster Indian Startup News" or poster_bio == "Promoted" or not poster or not poster_bio:
        continue
    profile = feed.find('a', {'class': 'update-components-actor__sub-description-link'})
    profile = profile["href"] if profile else profile;
    feed_text = feed.find('span', {'class': 'break-words tvm-parent-container'});

    hrefs = feed_text.find_all("a") if feed_text else []
    href_map = {}
    for href in hrefs:
        link_text = href.get_text() if href else ""
        if link_text.find("hashtag#") != -1 or link_text == "" :
            continue;
        href_map[href.get_text()] = href["href"] if href else href;
    post = {}
    post["poster"] = poster;
    post["poster_bio"] =  poster_bio;
    post["profile"] =  profile;
    post["feed_text"] =  feed_text.get_text() if feed_text else feed_text
    post["href_map"] = href_map;
    post["createdAt"] = datetime.utcnow()
    post["status"] = "pending"
    posts.append(post)
    print(post)
    print("\n")
    
    
    

{'poster': 'Akshat Shrivastava', 'poster_bio': 'Founder@Wisdom Hatch | Become a better investor --> WisdomHatch.com', 'profile': 'https://www.linkedin.com/in/akshatshrivastavainsead?miniProfileUrn=urn%3Ali%3Afsd_profile%3AACoAAAOluNsBmqekBgjJh6Oo0de70G0Zq7-_1Ok', 'feed_text': "\n Inflation is rate of change of prices. Not absolute increase of prices. Meaning: if the price goes from: [1] 100 to 110, that's 10% inflation [2] If it goes from 110 to 115, that's 4.5% inflationBut, realistically -- prices have gone up by 15%, yet the inflation is 4.5% Add to this on WHAT products: this entire 10%, 4.5% is calculated (i.e design of the inflation basket)--- inflation can be made to appear whatever you want it to appear. People don't understand all this complicated math/economics. And, this makes them keep wondering: inflation number is so low, but my prices are going high. How is this possible? Well, this is the explanation that you need to keep in mind :)\n", 'href_map': {}, 'createdAt': date

In [10]:

# # Extracting the HTML of the complete introduction box
# # that contains the name, company name, and the location
# feeds = soup.find_all('div', {'class': 'feed-shared-update-v2__description-wrapper'})
 

In [11]:
    
# feed_texts = [ feed.find('span', {'class': 'break-words tvm-parent-container'}) for feed in feeds]
# for feed_text in feed_texts:
#     print(feed_text.get_text())
#     print(feed_text.get_text().find("hiring"))
#     print("\n")

In [12]:

# Given text
# given_text = nlp("🚀 We're Hiring! Join Our Dynamic Team! We're excited to announce that [Company Name] is expanding, and we're looking for passionate individuals to join us. If you're someone who thrives in a fast-paced environment and loves solving challenging problems, we want to hear from you! Open Positions:Software EngineerLocation: [City, Country]Apply HereData ScientistLocation: [City, Country]Apply HereProduct ManagerLocation: [City, Country]Apply HereWhy [Company Name]?Competitive salary and benefitsOpportunity to work with a talented teamFlexible working hours and remote work optionsProfessional development and growth opportunitiesInterested?Click on the links above to apply or visit our Careers Page. Share this post with anyone who might be a great fit! Let's build something amazing together.#Hiring #JobOpportunity #JoinOurTeam #TechJobs")
# given_text2=nlp("I am a passionate Computer Engineering student with a minor in Artificial Intelligence, currently seeking full-time Software Engineering roles. I have experience in developing and optimizing software solutions, with strong skills in C/C++, SQL, JavaScript, ReactJs, Node.js, and database management (SQL, NoSQL). I am particularly interested in positions that involve: Backend development using Node.js and SQL/NoSQL databases. API development and optimization for scalable applications. Collaborative projects in agile environments. Opportunities for growth in machine learning and AI integration. I have a proven track record of success, including leading teams in hackathons and internships where I contributed to significant performance improvements. I am looking for a role where I can leverage my technical skills and leadership abilities to contribute to innovative projects. If your company is hiring for roles that match these interests, I would love to connect! #SoftwareEngineering #BackendDevelopment #NodeJS #AI #APIDevelopment #TechJobs #Hiring");
# # Filter texts based on similarity
# threshold = 0.7  # Set your threshold

# filtered_texts = [post for post in posts if post["feed_text"] and given_text.similarity(nlp(post["feed_text"])) > threshold]


# for feed_text in filtered_texts:
#     print(feed_text)
#     print("\n")

In [13]:
# SAVE THE DATA TO THE MONGO DB



In [14]:
db = client["linkedin-scrap"]
posts_collection = db["posts"]


In [15]:
posts_collection.insert_many(posts)

InsertManyResult([ObjectId('66c0340f518da520bfab3309'), ObjectId('66c0340f518da520bfab330a'), ObjectId('66c0340f518da520bfab330b'), ObjectId('66c0340f518da520bfab330c'), ObjectId('66c0340f518da520bfab330d'), ObjectId('66c0340f518da520bfab330e'), ObjectId('66c0340f518da520bfab330f'), ObjectId('66c0340f518da520bfab3310'), ObjectId('66c0340f518da520bfab3311'), ObjectId('66c0340f518da520bfab3312'), ObjectId('66c0340f518da520bfab3313'), ObjectId('66c0340f518da520bfab3314'), ObjectId('66c0340f518da520bfab3315'), ObjectId('66c0340f518da520bfab3316'), ObjectId('66c0340f518da520bfab3317'), ObjectId('66c0340f518da520bfab3318'), ObjectId('66c0340f518da520bfab3319'), ObjectId('66c0340f518da520bfab331a'), ObjectId('66c0340f518da520bfab331b'), ObjectId('66c0340f518da520bfab331c'), ObjectId('66c0340f518da520bfab331d'), ObjectId('66c0340f518da520bfab331e'), ObjectId('66c0340f518da520bfab331f'), ObjectId('66c0340f518da520bfab3320'), ObjectId('66c0340f518da520bfab3321'), ObjectId('66c0340f518da520bfab33

In [16]:
# for post in posts:
    